# Opentrons OT-2 protocol quick-start

This notebook is a tool for new Opentrons users developing their protocols using the Python API. It includes several custom functions that assist with accurate pipetting.

## Header info

If you make any significant modificiation to this file, please update the infomation in this cell...

**Version:** 2

**Committed by:** Alaric Taylor <alaric.taylor@ucl.ac.uk>
**Last modified by:** Yann Mamie <y.mamie@ucl.ac.uk>

**Commit date:** 2020-09-24

**Changelog (from last version):**
* First committ
* Added a wetting, 2 mix and 2 transfer functions

## Library imports

In [121]:
from opentrons import protocol_api

In [122]:
# Used within custom functions
import numpy as np
from scipy.interpolate import interp1d
import math
import time

## Metadata
Specifying `apiLevel` in the metadata is important... see [protocol API version control](https://docs.opentrons.com/v2/versioning.html) for more info.

In [123]:
metadata = {
    'apiLevel': '2.6', # maximum supported API level is visible in the Opentrons App
    'protocolName':'My Protocol',
    'description':'Simple protocol to get started using the OT-2',
    'author': 'Alaric Taylor'}

## References and tools
* Opentrons website
    * [API documention](https://docs.opentrons.com/)
    * [PDF reference guide](https://docs.opentrons.com/OpentronsPythonAPIV2.pdf)
* GitHub
    * [Library page](https://libraries.io/github/Opentrons/opentrons) (high-level)
    * [API](https://github.com/Opentrons/opentrons/tree/edge/api)
    * [API v2 source code](https://github.com/Opentrons/opentrons/tree/edge/api/docs/v2) (more detailed)
* PyPi (package distribution)
    * [PyPi](https://pypi.org/project/opentrons/)


### Labware
* Standard Opentrons labware [library](https://labware.opentrons.com/)
* [Labware creator](https://labware.opentrons.com/create/) (for custom labware defenitions)


### Markdown
 * Basic [syntax](https://www.markdownguide.org/basic-syntax/)
 * Wordpress [quick reference](https://wordpress.com/support/markdown-quick-reference/)

# Custom pipetting functions

These functions give fine control over the height at which an aspiration is made. This is implemented by tracking meniscus heights in each well. Thereby,
* maximising the accuracy of the pipette stroke (by pipetting at the top of a liquid)
* avoiding the formation of droplets along the outer pipette shaft
* preventing silent failures of a protocol e.g. if stock solution volumes are low and a pipette stroke fails to pick up liquid

In [124]:
def custom_aspirate(pipette, volume, location, tip_submersion_depth = 2, safety_height = 0.5):
    # aspiration_depth (in mm) beneath meniscus
    # safety_height prevents the tip from crashing into bottom of the well
    
    # Check a headroom attribute has been defined for the current location 
    if not hasattr(location, "headroom"): # If not, break 
        print('Headroom not set for ' + str(location))
        
    # Check a gradations attribute has been defined for the current location 
    if not hasattr(location, "volume_headroom_functions"): # If not, break 
        print('volume_headroom_functions (based upon gradations) not set for ' + str(location))
        
    # Unpack the vh functions that have been attributed to this tube
    v_given_h = location.volume_headroom_functions[0]
    h_given_v = location.volume_headroom_functions[1]
    
    
    final_volume = v_given_h(location.headroom) - volume
    # print('final_volume ', str(final_volume))
    depth_to_aspirate = h_given_v(final_volume) + tip_submersion_depth
    # print('depth_to_aspirate ', str(depth_to_aspirate))
    
    location.headroom = h_given_v(final_volume)
    
    # Safety checking
    if depth_to_aspirate > location._depth - safety_height: # e.g. 0.5mm above the bottom of the well
        depth_to_aspirate = location._depth - safety_height
        print('Warning, aspiration height very low...')
    
    if depth_to_aspirate < 0: #i.e. significantly above the well
        depth_to_aspirate = 0
    
    pipette.move_to(location.top(-depth_to_aspirate))
    pipette.aspirate(volume)
    


The `custom_dispense()` function record the volume of fluid added to a well.

In [125]:
def custom_dispense(pipette, volume, location):
    # Air dispensing
    
    # Check a headroom attribute has been defined for the current location 
    if not hasattr(location, "headroom"): # If not, break 
        print('Headroom not set for ' + str(location))
        
    # Check a gradations attribute has been defined for the current location 
    if not hasattr(location, "volume_headroom_functions"): # If not, break 
        print('volume_headroom_functions (based upon gradations) not set for ' + str(location))
        
    # Unpack the vh functions that have been attributed to this tube
    v_given_h = location.volume_headroom_functions[0]
    h_given_v = location.volume_headroom_functions[1]
    
    current_filled_volume = v_given_h(location.headroom)
    
    # Set new headroom from volume info
    location.headroom = h_given_v(current_filled_volume + volume)
    
    pipette.move_to(location.top())
    pipette.dispense(volume)

In [126]:
def custom_wetting(pipette, volume, location):
    custom_aspirate(pipette, volume, location)
    custom_dispense(pipette, volume, location)
    pipette.blow_out()
    pipette.touch_tip()

In [127]:
def custom_mixing_basic(pipette, volume, location, cycles, tip): # Using basic function, height changed for dispensing
    pipette.pick_up_tip(tip)
    for i in range(0, cycles):
        custom_aspirate(pipette, volume, location)
        custom_dispense(pipette, volume, location)
    pipette.blow_out()
    pipette.touch_tip()
    pipette.return_tip()

In [128]:
def custom_mixing_static(pipette, volume, location, cycles, tip, safety_height = 0.5): # Using mix function, static
    
    # Check a headroom attribute has been defined for the current location 
    if not hasattr(location, "headroom"): # If not, break 
        print('Headroom not set for ' + str(location))
        
    # Check a gradations attribute has been defined for the current location 
    if not hasattr(location, "volume_headroom_functions"): # If not, break 
        print('volume_headroom_functions (based upon gradations) not set for ' + str(location))
        
    # Unpack the vh functions that have been attributed to this tube
    v_given_h = location.volume_headroom_functions[0]
    h_given_v = location.volume_headroom_functions[1]
    
    
    intermediate_v = v_given_h(location.headroom) - volume
    mixing_h = (location.depth() - h_given_v(intermediate_v))/2          # Set mixing position to the middle of filled height
    
    if mixing_h <= safety_height:                         # Verify that there is a sufficient volume to perform the desired mixing
        print('Mixing volume is too high compared to the total volume, mixing cancelled')
    else:    
        pipette.pick_up_tip(tip)
        pipette.mix(cycles, volume, target.bottom(mixing_h))
        pipette.blow_out()
        pipette.touch_tip()
        pipette.return_tip()

In [129]:
def custom_transfer_simple(pipette, volume, source, destination, pipette_max_V):
    if volume <= pipette_max_V:                 # If the volume to transfer is lower than the pipette volume: simple transfer using custom functions
        custom_aspirate(pipette, volume, source)
        pipette.touch_tip()
        custom_dispense(pipette, volume+40, destination)
        pipette.blow_out()
    else:                                       # If the volume to transfer is higher than the pipette volume: several transfer steps at equal volume
        V_transfer = volume/math.ceil(volume/pipette_max_V)
        i = 0
        for i in range(0, math.ceil(volume/pipette_max_V)):
            custom_aspirate(pipette, V_transfer, source)
            pipette.touch_tip()
            custom_dispense(pipette, V_transfer+40, destination)
            pipette.blow_out()  

In [130]:
def custom_transfer_reverse(pipette, volume, source, destination, pipette_max_V, extra_V = 10):
    if volume <= pipette_max_V - extra_V:      # If the volume to transfer (including extra) is lower than the pipette volume: simple transfer using custom functions
        if volume + extra_V >= pipette_max_V:
            extra_V = pipette_max_V - volume
        custom_aspirate(pipette, volume+extra_V, source)
        pipette.touch_tip()
        custom_dispense(pipette, volume, destination)
        custom_dispense(pipette, extra_V+50, source)   # Ensures all the extra volume is returned to the source
        pipette.blow_out()
        pipette.touch_tip()
    else:                                       # If the volume to transfer is higher than the pipette volume: several transfer steps at equal volume
        transfer_number = math.ceil(volume/pipette_max_V)  # math.ceil rounds up to the next integer, this calculates the required number of transfer steps
        V_transfer = volume/transfer_number
        if V_transfer > pipette_max_V - extra_V:    # If the calculated transfer volume is too high to accomodate the extra volume, adds a step to guarantee all the volume can be transferred
            transfer_number += 1                    # The number of steps and volume are recalculated accordingly
            V_transfer = volume/transfer_number
        i = 0
        for i in range(0, transfer_number):         # Sequential addition of the desired volume using custom functions
            custom_aspirate(pipette, V_transfer+extra_V, source)
            pipette.touch_tip()
            custom_dispense(pipette, V_transfer, destination)
            custom_dispense(pipette, extra_V+50, source)   # Ensures all the extra volume is returned to the source
            pipette.blow_out()
            pipette.touch_tip()

Remaining functions (that need writing):
* Mixing
* Wetting
* Transfer (forward/standard)
* Transfer (reverse) e.g. for viscous liquids

Anticipated challenges:
* Pausing the protocol within a custom function is tricky as the `protocol` instance is not passed to the function. So `time.sleep()` has to be used instead...which impacts upon the simulation of the protocol.

## Attributes for meniscus height tracking

In [131]:
####################################
# Generate volume_headroom_functions
# for each type of tube in use from 
# measured gradations...

def gradations_to_vh(gradations):
    # Converts gradations (numpy array):
    # to tuple containg the v_given_h and h_given_v interpolation functions
    
    # Slicing the gradation array
    h = gradations[:,0] # headroom i.e. meniscus to top of container
    v = gradations[:,1] # fill volume
    
    maximum_headroom = gradations[:,0][-1]
    maximum_volume = gradations[:,1][0]
    
    # Interpolation functions
    # if kind='cubic', there is a risk the interpolated function
    # becomes negative close to the bottom of the well
    v_given_h = interp1d(h, v, kind='linear', assume_sorted=False, bounds_error=False, fill_value=(maximum_volume,0))
    h_given_v = interp1d(v, h, kind='linear', assume_sorted=False, bounds_error=False, fill_value=(maximum_headroom,0))
    
    # The order of the returned function is referenced by 
    # the custom_aspirate() and custom_dispense() functions
    return [v_given_h, h_given_v] 

Below, we define the gradation profiles for our standard tubes/wells. These are then converted to 'volume_headroom_functions' using `gradations_to_vh()`, above. The 'volume_headroom_functions' are then ascribed to each well instance at the beginning of the protocol.

In [132]:
#####################################
# Gradations defined as a numpy array 
# (cast from a list of sublists)
# Each sublost is structured as [meniscus_to_top_in_mm, uL_volume]

#####
# 1.5ml Eppendorph tubes
gradations = np.array([[1,1500], # Top gradation of container i.e. maximum liquid fill
                       [5,1000],
                       [10,500],
                       [13, 20],
                       [15,0]]) # Bottom of container i.e. no liquid

tube_vh = gradations_to_vh(gradations)

#####
# 15ml Falcon tubes


#####
# 50ml Falcon tubes


#####
# etc.



### OT-2 deck layout

|	|	|	|
|:--:	|:--:	|:-----:	|
| 10 	| 11 	| Trash 	|
|  7 	|  8 	|   9   	|
|  4 	|  5 	|   6   	|
|  1 	|  2 	|   3   	|

### Well referencing schema

| <blank>| <blank>| <blank>| <blank>| <blank>|
|:--:	|:--:	|:-----:	|:-----:	|:-----:	|
|<blank>	|**1**	|**2**	| **3** | … |
| **A**| <blank>| <blank>| <blank>| <blank>|
| **B** | <blank>| <blank>| <blank>| <blank>|
|  **C** | <blank>| <blank>| <blank>| <blank>|
|  ⋮  	| <blank>| <blank>| <blank>| <blank>|

# The Protocol

In [133]:
# protocol run function. the part after the colon lets your editor know
# where to look for autocomplete suggestions...but doesn't seem to work with Jupyter

def run(protocol: protocol_api.ProtocolContext):
    
    #############
    # Deck layout
    
    # Labware
    tiprack = protocol.load_labware('opentrons_96_tiprack_300ul',
                                        location='1',
                                        label='tiprack')
    tubes = protocol.load_labware('opentrons_24_aluminumblock_nest_1.5ml_screwcap',
                                        location='2',
                                        label='tubes')
    
    # Pipettes
    p50 = protocol.load_instrument('p50_single', 'left', tip_racks=[tiprack])
    p300 = protocol.load_instrument('p300_single', 'right', tip_racks=[tiprack])
    
    
    ######################
    # Named tubes and tips
    
    # Tips
    methanol_tip = tiprack['A1']
    water_tip = tiprack['A2']
    mixture_tip = tiprack['A3']

    # Tubes
    methanol = tubes['A1']
    water = tubes['A2']
    mixture = tubes['A3']
    
    # Setting instance attributes for 
    # all wells... simpler than 
    # extending classes and labware defenitions?
    #
    # Add for loop for each labware item
    for well in tubes.wells():
        well.headroom = well._depth # Initiated, but considered empty
        
        # Generate volume_headroom_functions uniques for
        # each well type by defining gradations and then 
        # running this through the gradations_to_vh() function, above
        well.volume_headroom_functions = tube_vh
    
    # Modify instance attributes for vials that start the protocol
    # filled with liquid...
    # Headroom measured in mm from meniscus to top of tube
    methanol.headroom = 5
    

    ####################
    ## General debugging
    # protocol.comment('Test comment')
    # protocol.pause('Test pause')
    
    ## Debugging instance attributes
    #attrs = vars(methanol)
    #print(str(attrs))
    
    #########
    # Actions
    
    p50.pick_up_tip(methanol_tip)
    p50.move_to(methanol.bottom(2)) # move to 2mm above the bottom of well A1
    custom_wetting(p50, 50, methanol)
    #custom_aspirate(p50, 50, methanol)
    custom_transfer_simple(p50, 50, methanol, mixture, 50)
    p50.return_tip()
    
    p50.pick_up_tip(water_tip)
    custom_wetting(p50, 50, water)
    custom_transfer_reverse(p50, 100, water, mixture, 50)
    p50.drop_tip()
    
    custom_mixing_basic(p50, 50, mixture, 2, mixture_tip)
    custom_mixing_static(p50, 50, mixture, 2, mixture_tip)
    
    #p50.drop_tip() # drop in trash
    
    protocol.pause()
    time.sleep(1)
    protocol.resume()

## Simulating protocol
1. Save as Python file (File > Download as...)
2. Inside python terminal, run:

`opentrons_simulate my_protocol.py`

Note, if using custom labware defenitions, append the following:

`--custom-labware-path="C:\Custom Labware"`
